# Task 3 model implementation

In [1]:
import numpy as np
import pandas as pd

from model import Model, Optimizer, NetworkModification, TransitScheduleModification

Creating a model implementation only requires pointing to a folder with all the necessary input data. Here, we have a region with 4 microtypes and four modes (walk, bus, auto, and rail), with rail only service two of the four microtypes

Once the model is created, you find supply/demand equilibrium and gather statistics such as mode splits, speeds, and user costs for an individual time period or for the whole day.

In [2]:
model= Model("input-data")

userCosts, operatorCosts = model.collectAllCosts()
modeSplit = model.getModeSplit('AM-Peak')
speeds = model.getModeSpeeds('AM-Peak')
print(modeSplit)
print(speeds)
print("Total user costs =", userCosts.total)

['rail: 0.3002507323611534', 'bus: 0.052142249598278644', 'auto: 0.6260030338362748', 'walk: 0.021603984204292895']
              A          B         C          D
rail  15.000000        NaN       NaN  20.000000
bus    7.583615   6.609926  6.708750   8.137854
auto  11.704067  10.227068  9.006314   9.806818
walk   1.400000   1.400000  1.400000   1.400000
Total user costs = 142723.73117455945


Once a model is created, there are a set of objects and methods that make it easy to modify the transportation system without needing to change the original input files.

Here, we will devote some road space to dedicated bus lanes. In particular, we are transitioning 1000 lane-meters of roadway from mixed bus/car traffic to bus-only lanes in microtypes A and B:

In [7]:
networkModification = NetworkModification(np.array([1000., 1000.]), list(zip([2, 4], [13, 14])))
model.modifyNetworks(networkModification = networkModification)

Once these changes are made, we can re-equilibrate the model and look at how metrics have changed. Here, we have (slightly) increased the bus mode share, in large part by increasing average bus speed in Microtypes A and B. However, because we have decreased auto speeds in these two microtypes, total user costs have gone up.

In [8]:
userCosts, operatorCosts = model.collectAllCosts()
modeSplit = model.getModeSplit('AM-Peak')
speeds = model.getModeSpeeds('AM-Peak')
print(modeSplit)
print(speeds)
print("Total user costs =", userCosts.total)

['walk: 0.02090659645903792', 'rail: 0.29754131019430363', 'bus: 0.06969772707237483', 'auto: 0.6118543662742836']
              A         B         C          D
walk   1.400000  1.400000  1.400000   1.400000
rail  15.000000       NaN       NaN  20.000000
bus    8.567903  7.152835  6.657390   8.139983
auto  11.197462  9.735467  9.196198   9.875834
Total user costs = 143588.21984194775


You can also modify the headways for the different transit modes. Here, we can decrease the headway of the bus mode in Microtype A to 200 seconds.

In [5]:
scheduleModification = TransitScheduleModification([200.0], [("A", "bus")])
model.modifyNetworks(scheduleModification = scheduleModification)
userCosts, operatorCosts = model.collectAllCosts()
modeSplit = model.getModeSplit('AM-Peak')
speeds = model.getModeSpeeds('AM-Peak')
print(modeSplit)
print(speeds)
print("Total user costs =", userCosts.total)

['walk: 0.021035070435338414', 'rail: 0.29795688960528594', 'bus: 0.06481848718333258', 'auto: 0.616189552776043']
              A         B         C          D
walk   1.400000  1.400000  1.400000   1.400000
rail  15.000000       NaN       NaN  20.000000
bus    7.858933  7.207186  6.674080   8.120795
auto  11.609340  9.660570  9.115997   9.834620
Total user costs = 143431.86439415868


Here, we can see that the bus mode share went up (because of lower wait times). In addition, bus speeds in microtype A increased because there was less crowding on each bus, but bus speeds slightly decreased in the other microtypes because of the increase in bus ridership without a corresponding decrease in headways.

In [6]:
print(model.getModeSpeeds('Overnight'))

              A          B          C          D
walk   1.400000   1.400000   1.400000   1.400000
rail  15.000000        NaN        NaN  20.000000
bus    9.325562  10.851088  11.227732  11.512197
auto  15.066836  15.020936  14.992872  14.883388
